In [4]:
import os
import json
import pandas as pd
import requests
from pandas.core.frame import DataFrame

In [8]:
df_pr1 = pd.read_csv('Data/all_runs_in_project_PRJEB11419.tsv', sep="	" , index_col=False)
df_QC1 = df_pr1.loc[df_pr1['QC status'] == 1.0]
df_phen_health = df_QC1.loc[df_QC1['associated phenotype'] == 'D006262'] #healthy
df_phen_health.head()

,project ID,run ID,experiment type,instrument model,nr. reads sequenced,host age,sex,BMI,country,longitude,lattitude,associated phenotype,QC status
18,PRJEB11419,ERR1072724,Amplicon,Illumina MiSeq,22818,66,Female,26.62,United States of America,-103.9,44.5,D006262,1.0
34,PRJEB11419,ERR1072851,Amplicon,Illumina MiSeq,22111,66,Female,24.20,United States of America,-71.5,41.4,D006262,1.0
53,PRJEB11419,ERR1073081,Amplicon,Illumina MiSeq,48612,30,Male,22.47,United States of America,-80.8,28.3,D006262,1.0
66,PRJEB11419,ERR1073129,Amplicon,Illumina MiSeq,33289,50,Male,27.77,Canada,-79.4,43.8,D006262,1.0
68,PRJEB11419,ERR1073205,Amplicon,Illumina MiSeq,42781,45,Female,20.94,Canada,0.0,0.0,D006262,1.0


In [17]:
obj_runs_healthy = df_phen_health['run ID']
#pd.DataFrame(df_runs_healthy, columns='Runs')
df_runs_healthy = obj_runs_healthy.to_frame()
df_runs_healthy.head()

,run ID
18,ERR1072724
34,ERR1072851
53,ERR1073081
66,ERR1073129
68,ERR1073205


In [18]:
all_flattened_runs = []

for index, run in df_runs_healthy['run ID'].items():
    query = {"run_id":run}  
    url = 'https://gmrepo.humangut.info/api/getFullTaxonomicProfileByRunID'
    data = requests.post(url, data=json.dumps(query)).json()

    ## --get run List
    run = data.get("run")

    ## --get DataFrames
    species = DataFrame(data.get("species"))
    species = species.drop(columns = ['taxon_rank_level', 'scientific_name']) #drop useless info

    species = species[species['ncbi_taxon_id'] != -1]   #drop missing taxons
    flat = species.set_index('ncbi_taxon_id')['relative_abundance'].T
    flat = flat.rename_axis(None).to_frame().T

    all_flattened_runs.append(flat)

final_df_health = pd.concat(all_flattened_runs, ignore_index=True)
final_df_health = pd.concat([final_df_health.reset_index(drop=True), final_df_health], axis=1)
final_df_health = final_df_health.fillna(0)
final_df_health['Depression'] = 0


C:\Users\Daria\AppData\Local\Temp\ipykernel_12648\2801113166.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df_health['Depression'] = 0


In [21]:
final_df_health.head()

,821,46506,820,376804,28117,204516,454154,28116,818,310297,...,1263979,261269,39765,1540221,1229,490759,1144748,664784,455360,Depression
0,9.341130,3.196960,2.122980,2.058040,1.288780,1.273790,1.158900,1.093960,0.724312,0.579450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,19.321400,0.052743,14.580400,0.011721,15.529800,0.023441,0.000000,1.359590,2.707450,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.020680,0.011489,0.022978,0.002298,0.000000,0.004596,0.000000,0.112592,0.075827,0.002298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,13.426000,0.026579,3.762770,0.003797,3.048940,0.015188,0.000000,3.071720,0.694840,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.258626,0.919885,5.936640,0.000000,0.649503,0.002939,0.288015,1.707520,3.611940,0.005878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
final_df_health.describe() # remove data with smallest coverage

,821,46506,820,376804,28117,204516,454154,28116,818,310297,...,1263979,261269,39765,1540221,1229,490759,1144748,664784,455360,Depression
count,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,...,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.0
mean,5.616084,1.266456,3.144276,0.046870,1.248025,1.020741,0.538026,2.186415,1.029484,0.450250,...,0.000003,0.000002,0.000002,0.000002,0.000002,0.000001,0.000001,0.000002,0.000003,0.0
std,7.755762,3.428096,3.868423,0.378752,1.549175,2.322615,1.569694,3.359996,1.500237,2.339425,...,0.000128,0.000090,0.000080,0.000082,0.000079,0.000055,0.000055,0.000077,0.000113,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.233385,0.023163,0.512154,0.000000,0.121232,0.013393,0.000000,0.342887,0.178806,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,2.205325,0.077439,1.831065,0.003435,0.713653,0.048649,0.004100,0.998101,0.524372,0.004704,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,8.105507,0.698991,4.286615,0.011666,1.814877,0.811333,0.166105,2.696710,1.291795,0.014841,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,56.732300,35.108800,33.604900,8.487010,15.529800,20.834500,23.561200,31.382900,13.941000,34.353200,...,0.004776,0.003374,0.002989,0.003042,0.002928,0.002034,0.002034,0.002854,0.004200,0.0


In [22]:
final_df_health.to_csv('Data/health_data_runs_abundance', index = False)

In [23]:
testdf = pd.read_csv('Data/health_data_runs_abundance')
testdf.head()

,821,46506,820,376804,28117,204516,454154,28116,818,310297,...,1263979.1,261269.1,39765.1,1540221.1,1229.1,490759.1,1144748.1,664784.1,455360.1,Depression
0,9.341130,3.196960,2.122980,2.058040,1.288780,1.273790,1.158900,1.093960,0.724312,0.579450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,19.321400,0.052743,14.580400,0.011721,15.529800,0.023441,0.000000,1.359590,2.707450,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.020680,0.011489,0.022978,0.002298,0.000000,0.004596,0.000000,0.112592,0.075827,0.002298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,13.426000,0.026579,3.762770,0.003797,3.048940,0.015188,0.000000,3.071720,0.694840,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.258626,0.919885,5.936640,0.000000,0.649503,0.002939,0.288015,1.707520,3.611940,0.005878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
